In [2]:
# Import neccessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplot
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import RandomizedPCA
from time import time
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
from time import time
from pandas import concat
from sklearn.preprocessing import Imputer

TARGET = "is_female"

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
print('---Load Data---')
t0 = time()
df = pd.read_csv('../WiDS/train.csv',low_memory=False)
test = pd.read_csv('../WiDS/test.csv',low_memory=False)
print('---Load Data Complete---')
print("done in %0.3fs" % (time() - t0))

---Load Data---
---Load Data Complete---
done in 9.565s


In [4]:
category_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print(len(category_cols))

96


In [5]:
TR_ROWS = df.shape[0]
print(df.shape)
print(test.shape)

(18255, 1235)
(27285, 1234)


In [6]:
df_all = pd.concat([df,test])
df_all.shape

(45540, 1236)

In [8]:
for header in category_cols:
#     df[header] = df[header].astype('category').cat.codes
#     test[header] = test[header].astype('category').cat.codes
    df_all[header] = df_all[header].astype('category').cat.codes.astype('int')
    df_all[header] = pd.to_numeric(df_all[header])
df_all.dtypes.value_counts()

float64    888
int64      333
object      15
dtype: int64

In [9]:
df =df_all.iloc[0:TR_ROWS]
test =df_all.iloc[TR_ROWS:]

df.drop(['test_id'],axis=1,inplace=True)
test.drop(['train_id',"is_female"],axis=1,inplace=True)

print(df.shape)
print(test.shape)

(18255, 1235)
(27285, 1234)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
df.dtypes.value_counts()

float64    887
int64      333
object      15
dtype: int64

In [11]:
test.dtypes.value_counts()

float64    886
int64      333
object      15
dtype: int64

In [12]:
drop_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
print(type(drop_cols))

<class 'list'>


In [15]:
df1 = df.drop(columns=drop_cols, axis=1)
print(type(df1))
print(df1.shape)
df1.dtypes.value_counts()

<class 'pandas.core.frame.DataFrame'>
(18255, 1220)


float64    887
int64      333
dtype: int64

In [16]:
test1 = test.drop(columns=drop_cols, axis=1)
print(type(test1))
print(test1.shape)
test1.dtypes.value_counts()

<class 'pandas.core.frame.DataFrame'>
(27285, 1219)


float64    886
int64      333
dtype: int64

In [17]:
df1.head()

,AA14,AA15,AA3,AA4,AA5,AA6,AA7,DG1,DG10b,DG10c,...,MT6C,MT6_OTHERS,MT7,MT7A,MT7A_OTHERS,MT8,MT9,MT9_OTHERS,is_female,train_id
0,3854,481,3,32,3.0,NaN,323011,1975,1.0,NaN,...,NaN,0,2.0,NaN,0,NaN,15.0,0,1.0,0.0
1,2441,344,2,26,NaN,8.0,268131,1981,1.0,NaN,...,NaN,0,1.0,3.0,0,2.0,NaN,0,1.0,1.0
2,754,143,1,16,NaN,7.0,167581,1995,2.0,0.0,...,NaN,0,1.0,1.0,0,2.0,NaN,0,1.0,2.0
3,5705,604,4,44,5.0,NaN,445071,1980,2.0,NaN,...,5.0,0,NaN,NaN,0,NaN,NaN,0,1.0,3.0
4,5645,592,4,43,NaN,6.0,436161,1958,NaN,NaN,...,NaN,0,2.0,NaN,0,NaN,3.0,0,1.0,4.0


In [18]:
test1.head()

,AA14,AA15,AA3,AA4,AA5,AA6,AA7,DG1,DG10b,DG10c,...,MT6B_OTHERS,MT6C,MT6_OTHERS,MT7,MT7A,MT7A_OTHERS,MT8,MT9,MT9_OTHERS,test_id
0,4479,535,4,41,NaN,7.0,417211,1979,1.0,NaN,...,0,NaN,0,2.0,NaN,0,NaN,12.0,0,0.0
1,3803,476,3,32,2.0,NaN,322011,1993,0.0,0.0,...,0,0.0,0,NaN,NaN,0,NaN,NaN,0,1.0
2,5610,585,3,36,5.0,NaN,365011,1980,1.0,NaN,...,0,NaN,0,2.0,NaN,0,NaN,13.0,0,2.0
3,2550,350,2,24,NaN,7.0,247061,1991,2.0,0.0,...,0,NaN,0,1.0,1.0,0,2.0,NaN,0,3.0
4,3233,400,3,35,NaN,8.0,358071,1985,NaN,NaN,...,0,NaN,0,2.0,NaN,0,NaN,15.0,0,4.0


In [23]:
df2=df1.drop([TARGET,'train_id'],axis=1)
y = df1[TARGET]
print(type(y))
print(df2.shape)
print(type(df2))
df2.head()

<class 'pandas.core.series.Series'>
(18255, 1218)
<class 'pandas.core.frame.DataFrame'>


,AA14,AA15,AA3,AA4,AA5,AA6,AA7,DG1,DG10b,DG10c,...,MT6B,MT6B_OTHERS,MT6C,MT6_OTHERS,MT7,MT7A,MT7A_OTHERS,MT8,MT9,MT9_OTHERS
0,3854,481,3,32,3.0,NaN,323011,1975,1.0,NaN,...,NaN,0,NaN,0,2.0,NaN,0,NaN,15.0,0
1,2441,344,2,26,NaN,8.0,268131,1981,1.0,NaN,...,NaN,0,NaN,0,1.0,3.0,0,2.0,NaN,0
2,754,143,1,16,NaN,7.0,167581,1995,2.0,0.0,...,NaN,0,NaN,0,1.0,1.0,0,2.0,NaN,0
3,5705,604,4,44,5.0,NaN,445071,1980,2.0,NaN,...,1.0,0,5.0,0,NaN,NaN,0,NaN,NaN,0
4,5645,592,4,43,NaN,6.0,436161,1958,NaN,NaN,...,NaN,0,NaN,0,2.0,NaN,0,NaN,3.0,0


In [24]:
test2 = test1.drop('test_id', axis=1)
print(test1.shape)
test2.head()

(27285, 1219)


,AA14,AA15,AA3,AA4,AA5,AA6,AA7,DG1,DG10b,DG10c,...,MT6B,MT6B_OTHERS,MT6C,MT6_OTHERS,MT7,MT7A,MT7A_OTHERS,MT8,MT9,MT9_OTHERS
0,4479,535,4,41,NaN,7.0,417211,1979,1.0,NaN,...,NaN,0,NaN,0,2.0,NaN,0,NaN,12.0,0
1,3803,476,3,32,2.0,NaN,322011,1993,0.0,0.0,...,1.0,0,0.0,0,NaN,NaN,0,NaN,NaN,0
2,5610,585,3,36,5.0,NaN,365011,1980,1.0,NaN,...,NaN,0,NaN,0,2.0,NaN,0,NaN,13.0,0
3,2550,350,2,24,NaN,7.0,247061,1991,2.0,0.0,...,NaN,0,NaN,0,1.0,1.0,0,2.0,NaN,0
4,3233,400,3,35,NaN,8.0,358071,1985,NaN,NaN,...,NaN,0,NaN,0,2.0,NaN,0,NaN,15.0,0


In [25]:
# 至此，对categorical feature的处理结束。进行下一步：对missing value的处理。
# 还是要先concatenate df2 and test2
total = pd.concat([df2, test2])
print(total.shape)
print(type(total))

(45540, 1218)
<class 'pandas.core.frame.DataFrame'>


In [26]:
cols_to_drop = [col for col in total.columns 
                                 if total[col].isnull().sum() > len(total)/2]
len(cols_to_drop)

764

In [27]:
total_drop = total.drop(cols_to_drop, axis=1)
print(total_drop.shape)
print(type(total_drop))
total_drop.head()

(45540, 454)
<class 'pandas.core.frame.DataFrame'>


,AA14,AA15,AA3,AA4,AA6,AA7,DG1,DG13_OTHERS,DG14_OTHERS,DG3,...,MT18_OTHERS,MT1A,MT2,MT5_OTHERS,MT6A_OTHERS,MT6B_OTHERS,MT6_OTHERS,MT7,MT7A_OTHERS,MT9_OTHERS
0,3854,481,3,32,NaN,323011,1975,0,0,3,...,0,99.0,2,0,0,0,0,2.0,0,0
1,2441,344,2,26,8.0,268131,1981,0,0,8,...,0,4.0,2,0,0,0,0,1.0,0,0
2,754,143,1,16,7.0,167581,1995,0,0,3,...,0,2.0,2,0,0,0,0,1.0,0,0
3,5705,604,4,44,NaN,445071,1980,0,0,3,...,0,2.0,1,0,0,0,0,NaN,0,0
4,5645,592,4,43,6.0,436161,1958,0,0,3,...,0,NaN,2,0,0,0,0,2.0,0,0


In [31]:
# SECOND STEP: imputation
my_imputer = Imputer(strategy='most_frequent')
total_imputed = my_imputer.fit_transform(total_drop)
print(total_imputed.shape)
print(type(total_imputed))

(45540, 454)
<class 'numpy.ndarray'>


In [32]:
total_imputed_df = pd.DataFrame(total_imputed, columns = total_drop.columns)
print(total_imputed_df.shape)
print(type(total_imputed_df))
total_imputed_df.head()

(45540, 454)
<class 'pandas.core.frame.DataFrame'>


,AA14,AA15,AA3,AA4,AA6,AA7,DG1,DG13_OTHERS,DG14_OTHERS,DG3,...,MT18_OTHERS,MT1A,MT2,MT5_OTHERS,MT6A_OTHERS,MT6B_OTHERS,MT6_OTHERS,MT7,MT7A_OTHERS,MT9_OTHERS
0,3854.0,481.0,3.0,32.0,6.0,323011.0,1975.0,0.0,0.0,3.0,...,0.0,99.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,2441.0,344.0,2.0,26.0,8.0,268131.0,1981.0,0.0,0.0,8.0,...,0.0,4.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,754.0,143.0,1.0,16.0,7.0,167581.0,1995.0,0.0,0.0,3.0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5705.0,604.0,4.0,44.0,6.0,445071.0,1980.0,0.0,0.0,3.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,5645.0,592.0,4.0,43.0,6.0,436161.0,1958.0,0.0,0.0,3.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [33]:
total_imputed_df.dtypes.value_counts()

float64    454
dtype: int64

In [34]:
total_final = total_imputed_df.astype('int')
print(total_final.shape)
print(type(total_final))
total_final.head()

(45540, 454)
<class 'pandas.core.frame.DataFrame'>


,AA14,AA15,AA3,AA4,AA6,AA7,DG1,DG13_OTHERS,DG14_OTHERS,DG3,...,MT18_OTHERS,MT1A,MT2,MT5_OTHERS,MT6A_OTHERS,MT6B_OTHERS,MT6_OTHERS,MT7,MT7A_OTHERS,MT9_OTHERS
0,3854,481,3,32,6,323011,1975,0,0,3,...,0,99,2,0,0,0,0,2,0,0
1,2441,344,2,26,8,268131,1981,0,0,8,...,0,4,2,0,0,0,0,1,0,0
2,754,143,1,16,7,167581,1995,0,0,3,...,0,2,2,0,0,0,0,1,0,0
3,5705,604,4,44,6,445071,1980,0,0,3,...,0,2,1,0,0,0,0,2,0,0
4,5645,592,4,43,6,436161,1958,0,0,3,...,0,1,2,0,0,0,0,2,0,0


In [35]:
total_final.dtypes.value_counts()

int64    454
dtype: int64

In [36]:
df_final = total_final.iloc[0:TR_ROWS]
test_final = total_final.iloc[TR_ROWS:]

In [38]:
print(df_final.shape)
df_final.head()

(18255, 454)


,AA14,AA15,AA3,AA4,AA6,AA7,DG1,DG13_OTHERS,DG14_OTHERS,DG3,...,MT18_OTHERS,MT1A,MT2,MT5_OTHERS,MT6A_OTHERS,MT6B_OTHERS,MT6_OTHERS,MT7,MT7A_OTHERS,MT9_OTHERS
0,3854,481,3,32,6,323011,1975,0,0,3,...,0,99,2,0,0,0,0,2,0,0
1,2441,344,2,26,8,268131,1981,0,0,8,...,0,4,2,0,0,0,0,1,0,0
2,754,143,1,16,7,167581,1995,0,0,3,...,0,2,2,0,0,0,0,1,0,0
3,5705,604,4,44,6,445071,1980,0,0,3,...,0,2,1,0,0,0,0,2,0,0
4,5645,592,4,43,6,436161,1958,0,0,3,...,0,1,2,0,0,0,0,2,0,0


In [39]:
print(test_final.shape)
test_final.head()

(27285, 454)


,AA14,AA15,AA3,AA4,AA6,AA7,DG1,DG13_OTHERS,DG14_OTHERS,DG3,...,MT18_OTHERS,MT1A,MT2,MT5_OTHERS,MT6A_OTHERS,MT6B_OTHERS,MT6_OTHERS,MT7,MT7A_OTHERS,MT9_OTHERS
18255,4479,535,4,41,7,417211,1979,0,0,8,...,0,2,2,0,0,0,0,2,0,0
18256,3803,476,3,32,6,322011,1993,0,0,1,...,0,1,1,0,0,0,0,2,0,0
18257,5610,585,3,36,6,365011,1980,0,0,3,...,0,2,2,0,0,0,0,2,0,0
18258,2550,350,2,24,7,247061,1991,0,0,3,...,0,2,2,0,0,0,0,1,0,0
18259,3233,400,3,35,8,358071,1985,0,0,3,...,0,1,2,0,0,0,0,2,0,0


In [41]:
# 啊啊啊，终于把preprocessing做完了
print(y.shape)

(18255,)


In [42]:
# split df_new to training and validation
X_train, X_test, y_train, y_test =train_test_split(df_final, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(14604, 454) (3651, 454) (14604,) (3651,)


In [60]:
# Random Forest
print ("---Random Forest Model---")
rf = RandomForestClassifier(n_estimators=500,max_depth=28, random_state=123)
rf.fit(X_train, y_train)
y_prob = rf.predict_proba(X_test)
roc_auc_score(y_test, y_prob[:,1])

---Random Forest Model---


0.95632952827390649

In [63]:
# testid = test['test_id'].astype('int')
# print(type(testid))
# print(testid.shape)
# testid.head()

In [66]:
# choose the best hyper-parameters to train whole training set
rf2 = RandomForestClassifier(n_estimators=500, max_depth=28, random_state=123)
rf2.fit(df_final, y)
y_submit = rf2.predict_proba(test_final)[:,1]

test = pd.read_csv('../WiDS/test.csv',low_memory=False)
test['is_female'] = y_submit
ans = test[['test_id','is_female']]
print(ans.shape)
ans.head()

(27285, 2)


,test_id,is_female
0,0,0.967928
1,1,0.245368
2,2,0.916000
3,3,0.915953
4,4,0.280000


In [68]:
ans.to_csv('submit4.csv', index=None)